In [ ]:
# Import all necessary python packages
import boto3
from botocore.exceptions import ClientError
from google.cloud import vision, storage
from google.oauth2 import service_account
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient, StandardBlobTier, PublicAccess
import torch
import io, json
import cv2
import os, shutil
import random
import time
from ultralytics import YOLO
import matplotlib.pyplot as plt
from datetime import datetime
import json
import numpy as np
from IPython.display import Image
import psutil, gpustat, cpuinfo
import PIL

yolov5_model = YOLO('yolov5s.pt')
yolov8_model = YOLO('yolov8s.pt')

In [ ]:
# Specify the folder containing the images and bbox data
#dataset_name = "Roboflow, Self Driving Car Image Dataset"
#original_image_folder = "C:\\Users\\feren\\Downloads\\Self Driving Car.v2-fixed-large.coco\\OUTPUT\\train"

dataset_name = "NuImages"
original_image_folder = "C:\\Users\\feren\\Downloads\\nuimages_output\\train"
image_folder = "temp_images"
output_path = "G:\\My Drive\\Thesis2024\\output"

# Specify the desired Y-axis resolution
desired_y_resolution = 900 #480 = VGA, 1200 = max

inference_limit = 5
request_time_limiter_ms = 300
input_images_number = 5

GOOGLE_MAX_RESULTS = 100 # no threshold setting
REKOGNITION_CONFIDENCE_THRESHOLD = 0 # in percentage
YOLOV5_CONFIDENCE_THRESHOLD = 0.0
YOLOV8_CONFIDENCE_THRESHOLD = 0.0

In [ ]:
# Change image resolution

# Delete the image_folder and its contents if it exists
if os.path.exists(image_folder):
    shutil.rmtree(image_folder)
# Create the image_folder directory if it doesn't exist
os.makedirs(image_folder, exist_ok=True)
# Iterate over the images in the original folder
for image_file in os.listdir(original_image_folder):
    # Check if the file is an image
    if image_file.endswith(".png") or image_file.endswith(".jpg"):
        # Open the image using PIL
        image_path = os.path.join(original_image_folder, image_file)
        image = PIL.Image.open(image_path)

        # Get the original image dimensions
        original_width, original_height = image.size

        # Calculate the new width while maintaining the aspect ratio
        new_width = int((desired_y_resolution / original_height) * original_width)

        # Resize the image
        resized_image = image.resize((new_width, desired_y_resolution))

        # Save the resized image to the image_folder
        out_p = os.path.join(image_folder, image_file)
        resized_image.save(out_p)
    elif image_file.endswith(".json"):
        # Copy the JSON file to the image_folder
        shutil.copy2(os.path.join(original_image_folder, image_file), image_folder)

In [ ]:
# Set up all API clients

# Amazon Rekognition
rekognition_client = boto3.client(
    "rekognition",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)

# Google Vision and Cloud Storage
credentials = service_account.Credentials.from_service_account_file(GOOGLE_KEY_PATH)
google_client = vision.ImageAnnotatorClient(credentials=credentials)
storage_client = storage.Client(credentials=credentials, project=GOOGLE_PROJECT_ID)

# Microsoft Azure
azure_client = ComputerVisionClient(MICROSOFT_ENDPOINT, CognitiveServicesCredentials(MICROSOFT_SUBSCRIPTION_KEY))


In [ ]:
# Amazon Rekognition API
def detect_objects_with_rekognition(image_path: str) -> dict:
    bucket_name = AWS_BUCKET_NAME
    object_key = os.path.basename(image_path)
    
    start_upload_time = time.time()
    s3_client.upload_file(image_path, bucket_name, object_key)
    upload_time = (time.time() - start_upload_time) * 1000  # Convert to milliseconds

    try:
        start_time = time.time()
        response = rekognition_client.detect_labels(
            Image={"S3Object": {"Bucket": bucket_name, "Name": object_key}},
            MinConfidence=REKOGNITION_CONFIDENCE_THRESHOLD
        )
        inference_time = (time.time() - start_time) * 1000  # Convert to milliseconds
        response["inference_time"] = inference_time
        response["upload_time"] = upload_time
        return response
    except ClientError as e:
        print("Error: {}".format(e))
        return {}

def convert_rekognition_output(rekognition_output: dict, image_id: str) -> dict:
    objects = []

    for label in rekognition_output.get("Labels", []):
        for instance in label.get("Instances", []):
            bbox = instance["BoundingBox"]
            objects.append({
                "class_name": label["Name"],
                "bbox": [bbox["Left"], bbox["Top"], bbox["Width"], bbox["Height"]],
                "confidence": instance["Confidence"] / 100
            })

    return {
        "image_id": image_id,
        "objects": objects,
        "inference_time": rekognition_output["inference_time"],
        "upload_time": rekognition_output["upload_time"]
    }

In [ ]:
# Google Vision
def detect_objects_with_google_vision(image_path: str) -> dict:
    bucket_name = GOOGLE_BUCKET_NAME  # export this later
    blob_name = os.path.basename(image_path)

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    start_upload_time = time.time()
    blob.upload_from_filename(image_path)
    upload_time = (time.time() - start_upload_time) * 1000  # Convert to milliseconds

    image = vision.Image(source=vision.ImageSource(gcs_image_uri=f"gs://{bucket_name}/{blob_name}"))

    start_time = time.time()
    response = google_client.object_localization(
        image=image,
        max_results=GOOGLE_MAX_RESULTS, 
    )
    inference_time = (time.time() - start_time) * 1000  # Convert to milliseconds
    
    return {"response": response, "inference_time": inference_time, "upload_time": upload_time}

def convert_google_vision_output(google_vision_output: dict, image_id: str) -> dict:
    response = google_vision_output["response"]
    objects = []

    for object_ in response.localized_object_annotations:
        vertices = object_.bounding_poly.normalized_vertices
        objects.append({
            "class_name": object_.name,
            "bbox": [vertices[0].x, vertices[0].y, vertices[2].x - vertices[0].x, vertices[2].y - vertices[0].y],
            "confidence": object_.score
        })

    return {
        "image_id": image_id,
        "objects": objects,
        "inference_time": google_vision_output["inference_time"],
        "upload_time": google_vision_output["upload_time"]
    }


In [ ]:
# Microsoft Azure
def detect_objects_with_azure(image_path: str) -> dict:
    container_name = AZURE_CONTAINER_NAME
    
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(container_name)

    blob_name = os.path.basename(image_path)

    start_upload_time = time.time()
    with open(image_path, "rb") as data:
        blob_client = container_client.get_blob_client(blob_name)
        blob_client.upload_blob(data, overwrite=True)
        container_client.set_container_access_policy(signed_identifiers={}, public_access=PublicAccess.Blob)  # Set public access level
    upload_time = (time.time() - start_upload_time) * 1000  # Convert to milliseconds

    image_url = f"https://{blob_service_client.account_name}.blob.core.windows.net/{container_name}/{blob_name}"

    start_time = time.time()
    analysis = azure_client.analyze_image(
        image_url,
        visual_features=[VisualFeatureTypes.objects],
        detection_model="detection_03",  # Use the latest detection model
    )
    inference_time = (time.time() - start_time) * 1000  # Convert to milliseconds

    # Delete the blob after inference
    container_client.delete_blob(blob_name)

    return {"analysis": analysis.as_dict(), "inference_time": inference_time, "upload_time": upload_time}


def convert_azure_output(azure_output: dict, image_path: str) -> dict:
    # Read the image using OpenCV
    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    objects = []
    for object_ in azure_output["analysis"].get("objects", []):
        bbox = object_["rectangle"]
        normalized_bbox = [
            bbox["x"] / image_width,
            bbox["y"] / image_height,
            bbox["w"] / image_width,
            bbox["h"] / image_height
        ]
        objects.append({
            "class_name": object_["object_property"],
            "bbox": normalized_bbox,
            "confidence": object_["confidence"]
        })

    return {
        "image_id": image_path,
        "objects": objects,
        "inference_time": azure_output["inference_time"],
        "upload_time": azure_output["upload_time"]
    }


In [ ]:
# YOLOv5
def detect_objects_with_yolov5(image_path: str) -> dict:
    image = cv2.imread(image_path)
    image.shape[:2]
    results = yolov5_model(image_path, conf=YOLOV8_CONFIDENCE_THRESHOLD, imgsz=image.shape[:2])

    return {
        "results": results,
        "inference_time": results[0].speed['inference']  # Retrieve inference time from YOLOv5 output
    }

def convert_yolov5_output(yolov5_output: dict, image_path: str) -> dict:
    objects = []

    # Get the image dimensions
    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    for result in yolov5_output["results"]:
        for object_ in result.boxes.data.cpu().numpy():
            xyxy = object_[:4]
            conf = object_[4]
            class_id = int(object_[5])

            # Normalize the bounding box coordinates
            normalized_bbox = [
                xyxy[0] / image_width,
                xyxy[1] / image_height,
                (xyxy[2] - xyxy[0]) / image_width,
                (xyxy[3] - xyxy[1]) / image_height
            ]

            objects.append({
                "class_name": result.names[class_id],
                "bbox": normalized_bbox,
                "confidence": conf
            })

    return {
        "image_id": image_path,
        "objects": objects,
        "inference_time": yolov5_output["inference_time"],
        "upload_time": 0  # Add a default upload time of 0 for local solutions
    }

In [ ]:
# YOLOv8
def detect_objects_with_yolov8(image_path: str) -> dict:
    image = cv2.imread(image_path)

    results = yolov8_model(image_path, conf=YOLOV8_CONFIDENCE_THRESHOLD, imgsz=image.shape[:2])

    return {
        "results": results,
        "inference_time": results[0].speed['inference']  # Retrieve inference time from YOLOv8 output
    }

def convert_yolov8_output(yolov8_output: dict, image_path: str) -> dict:
    objects = []

    # Get the image dimensions
    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    for result in yolov8_output["results"]:
        for object_ in result.boxes.data.cpu().numpy():
            xyxy = object_[:4]
            conf = object_[4]
            class_id = int(object_[5])

            # Normalize the bounding box coordinates
            normalized_bbox = [
                xyxy[0] / image_width,
                xyxy[1] / image_height,
                (xyxy[2] - xyxy[0]) / image_width,
                (xyxy[3] - xyxy[1]) / image_height
            ]

            objects.append({
                "class_name": result.names[class_id],
                "bbox": normalized_bbox,
                "confidence": conf
            })

    return {
        "image_id": image_path,
        "objects": objects,
        "inference_time": yolov8_output["inference_time"],
        "upload_time": 0  # Add a default upload time of 0 for local solutions
    }


In [ ]:
def sort_bounding_boxes(output_dict):
    objects = output_dict["objects"]
    objects.sort(key=lambda x: x["bbox"][2] * x["bbox"][3], reverse=True)
    return output_dict

In [ ]:
# One function to run them all
def inferenceFromAllSources(image_path: str) -> dict:
    google_vision_out = convert_google_vision_output(detect_objects_with_google_vision(image_path), image_path)
    azure_out = convert_azure_output(detect_objects_with_azure(image_path), image_path)
    rekognition_out = convert_rekognition_output(detect_objects_with_rekognition(image_path), image_path)
    yolov5_out = convert_yolov5_output(detect_objects_with_yolov5(image_path), image_path)
    yolov8_out = convert_yolov8_output(detect_objects_with_yolov8(image_path), image_path)
    ##yolov9_out = convert_yolov9_output(detect_objects_with_yolov9(image_path), image_path)

    return {
        "google": sort_bounding_boxes(google_vision_out),
        "azure": sort_bounding_boxes(azure_out),
        "rekognition": sort_bounding_boxes(rekognition_out),
        "yolov5": sort_bounding_boxes(yolov5_out),
        "yolov8": sort_bounding_boxes(yolov8_out),
        ##"yolov9": sort_bounding_boxes(yolov9_out),
    }

In [ ]:
def print_dict(data, indent_level=0):
    indent = "  " * indent_level
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict):
                print(f"{indent}{key}:")
                print_dict(value, indent_level + 1)
            elif isinstance(value, list) and all(isinstance(item, dict) for item in value):
                print(f"{indent}{key}:")
                for item in value:
                    print(f"{indent}  -")
                    print_dict(item, indent_level + 2)
            else:
                print(f"{indent}{key}: {value}")
    else:
        print(f"{indent}{data}")

In [ ]:
def draw_bounding_boxes(image_path, objects):
    image = cv2.imread(image_path)

    # Create a dictionary to store colors for each class
    class_colors = {}

    for obj in objects:
        class_name = obj["class_name"]
        bbox = obj["bbox"]

        # Generate a random color for each class if not already assigned
        if class_name not in class_colors:
            class_colors[class_name] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        color = class_colors[class_name]

        # Convert normalized coordinates to pixel coordinates
        height, width, _ = image.shape
        bbox = [int(bbox[0] * width), int(bbox[1] * height), int(bbox[2] * width), int(bbox[3] * height)]

        # Draw bounding box rectangle
        cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), color, 2)

        # Put class name text above the rectangle
        cv2.putText(image, class_name, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Convert the image to a format compatible with display()
    _, encoded_image = cv2.imencode('.png', image)
    display(Image(data=encoded_image.tobytes()))

In [ ]:
# This gets the image file names, necessary to run
# Load the JSON file containing the bbox data
with open(os.path.join(image_folder, "labels.json"), "r") as file:
    original_bbox_data = json.load(file)

# Get a list of image file names
image_files = [file for file in os.listdir(image_folder) if file.endswith(".png") or file.endswith(".jpg")]

# Randomly select an image file
selected_image = random.choice(image_files)
image_path = os.path.join(image_folder, selected_image)

# Find the corresponding bbox data for the selected image
selected_bbox_data = next((data for data in original_bbox_data if data["image_id"] == selected_image), None)
image = cv2.imread(image_path)

# Convert the image to a format compatible with display()
_, encoded_image = cv2.imencode('.png', image)
display(Image(data=encoded_image.tobytes()))

In [ ]:
def convert_class_names(objects):
    # Dictionary to map alternative class names to a common class name
    class_mapping = {
        "car": ["automobile", "taxi", "vehicle", "suv", "jeep", "sedan", "van", "land vehicle", "vehicle.car", "vehicle.emergency.police"],
        "truck": ["truck", "lorry", "bus", "shuttle bus", "pickup truck", "vehicle.truck", "vehicle.bus.bendy", "vehicle.bus.rigid", "vehicle.trailer", "vehicle.construction", "vehicle.emergency.ambulance"],
        "person": ["person", "pedestrian", "human", 
                   "human.pedestrian.adult", "human.pedestrian.child", "human.pedestrian.construction_worker", 
                   "human.pedestrian.personal_mobility", "human.pedestrian.police_officer", "human.pedestrian.stroller", 
                   "human.pedestrian.wheelchair"],
        "biker": ["bicycle", "bike", "biker", "motorcycle", "motorbike", "vehicle.bicycle", "vehicle.motorcycle"],
        # Add more mappings as needed
    }

    converted_objects = []
    for obj in objects:
        class_name = obj["class_name"].lower()  # Convert class name to lowercase
        obj["class_name"] = class_name  # Store the lowercase class name back in the original object
        for common_name, alt_names in class_mapping.items():
            if class_name == common_name or class_name in alt_names:
                obj["class_name"] = common_name
                converted_objects.append(obj)
                break
        # Remove the following line to delete objects not found in the class_mapping dictionary:
        # else: converted_objects.append(obj)

    return converted_objects


In [ ]:
def convert_to_pixel_coords(bbox, image_width, image_height):
    x, y, w, h = bbox
    x_min = int(x * image_width)
    y_min = int(y * image_height)
    x_max = int((x + w) * image_width)
    y_max = int((y + h) * image_height)
    return x_min, y_min, x_max, y_max

def create_class_masks(objects, image_width, image_height):
    class_masks = {}
    for obj in objects:
        class_name = obj['class_name']
        if class_name not in class_masks:
            class_masks[class_name] = np.zeros((image_height, image_width), dtype=int)
        x_min, y_min, x_max, y_max = convert_to_pixel_coords(obj['bbox'], image_width, image_height)
        class_masks[class_name][y_min:y_max, x_min:x_max] += 1
    return class_masks

def calculate_iou(inferred_objects, original_objects, image_width, image_height):
    inferred_masks = create_class_masks(inferred_objects, image_width, image_height)
    original_masks = create_class_masks(original_objects, image_width, image_height)

    total_intersection = 0
    total_union = 0

    # Calculate intersection for each class
    for class_name in inferred_masks:
        if class_name in original_masks:
            intersection = np.minimum(inferred_masks[class_name], original_masks[class_name])
            total_intersection += np.sum(intersection)

    # Calculate union across all classes
    all_classes = set(inferred_masks.keys()).union(set(original_masks.keys()))
    for class_name in all_classes:
        inferred_mask = inferred_masks.get(class_name, np.zeros((image_height, image_width), dtype=int))
        original_mask = original_masks.get(class_name, np.zeros((image_height, image_width), dtype=int))
        union = np.maximum(inferred_mask, original_mask)
        total_union += np.sum(union)

    iou = total_intersection / total_union if total_union > 0 else 0
    return iou

In [ ]:
#----RUN INFERENCE SEQUENTIALLY ON IMAGES

# List to store the inference results for all images
all_inference_results = []

# Randomly select input_images_number of images from image_files
selected_images = random.sample(image_files, input_images_number)

# Initialize variables for failsafe and time limiting
inference_count = 0
last_request_time = 0

# Iterate over the selected images
for image_file in selected_images:
    # Check if the inference limit has been reached
    if inference_count >= inference_limit:
        print("Inference limit reached. Stopping the loop.")
        break

    # Check if enough time has passed since the last request
    current_time = time.time()
    elapsed_time = (current_time - last_request_time) * 1000  # Convert to milliseconds
    if elapsed_time < request_time_limiter_ms:
        # Wait for the remaining time to reach the request time limit
        time.sleep((request_time_limiter_ms - elapsed_time) / 1000)

    # Update the last request time
    last_request_time = time.time()

    # Construct the full image path
    image_path = os.path.join(image_folder, image_file)

    try:
        # Call the inferenceFromAllSources function for the current image
        inference_result = inferenceFromAllSources(image_path)

        # Append the inference result to the list
        all_inference_results.append(inference_result)

        # Increment the inference count
        inference_count += 1

        print(f"Inference completed for {image_file}. Inference count: {inference_count}")
    except Exception as e:
        print(f"Error occurred during inference for {image_file}: {str(e)}")
        continue

# Print the number of images processed
print(f"Inference completed for {len(all_inference_results)} images.")
#print_dict(all_inference_results)

In [ ]:
def find_optimal_confidence_thresholds(inference_results, bbox_data, threshold_step=0.02):
    optimal_thresholds = {}
    image_dimensions = {}
    bbox_data_dict = {data["image_id"]: data for data in bbox_data}
    
    # Precompute image dimensions
    for inference_result in inference_results:
        for method in inference_result:
            image_id = inference_result[method]["image_id"]
            if image_id not in image_dimensions:
                image = cv2.imread(image_id)
                if image is not None:
                    height, width = image.shape[:2]
                    image_dimensions[image_id] = (width, height)

    for method in inference_results[0].keys():
        max_avg_iou = 0
        optimal_threshold = 0
        
        thresholds = np.arange(0, 1 + threshold_step, threshold_step)
        total_ious = np.zeros(thresholds.shape)
        image_counts = np.zeros(thresholds.shape)

        for inference_result in inference_results:
            image_id = inference_result[method]["image_id"]
            bbox_data_item = bbox_data_dict.get(os.path.basename(image_id))
            if bbox_data_item:
                width, height = image_dimensions.get(image_id, (None, None))
                for i, threshold in enumerate(thresholds):
                    filtered_objects = [obj for obj in inference_result[method]["objects"] if obj["confidence"] >= threshold]
                    image_iou = calculate_iou(filtered_objects, bbox_data_item["objects"], width, height)
                    total_ious[i] += image_iou
                    image_counts[i] += 1

        avg_ious = np.divide(total_ious, image_counts, out=np.zeros_like(total_ious), where=image_counts != 0)
        max_avg_iou_index = np.argmax(avg_ious)
        optimal_threshold = thresholds[max_avg_iou_index]

        optimal_thresholds[method] = round(optimal_threshold, 2)

    return optimal_thresholds


In [ ]:
# Converts class names and calculates optimal confidence thresholds for all images, also loads original bbox data

# Load the JSON file containing the bbox data
with open(os.path.join(image_folder, "labels.json"), "r") as file:
    original_bbox_data = json.load(file)

def convert_class_names_in_results(inference_results, bbox_data):
    bbox_data_dict = {data["image_id"]: data for data in bbox_data}

    for inference_result in inference_results:
        for method in inference_result:
            inference_result[method]["objects"] = convert_class_names(inference_result[method]["objects"])
        
        image_file = os.path.basename(inference_result["yolov5"]["image_id"])
        bbox_data_item = bbox_data_dict.get(image_file)
        
        if bbox_data_item:
            bbox_data_item["objects"] = convert_class_names(bbox_data_item["objects"])
    
    return inference_results, list(bbox_data_dict.values())


# Convert class names in the inference results and original data
all_inference_results, original_bbox_data = convert_class_names_in_results(all_inference_results, original_bbox_data)

optimal_thresholds = find_optimal_confidence_thresholds(all_inference_results, original_bbox_data)
print(optimal_thresholds)


In [ ]:
# Creates a new version of all_inference_results with bboxes below the confidence threshold removed
optimized_inference_results = []

for inference_result in all_inference_results:
    optimized_result = {}
    for method in inference_result:
        optimal_threshold = optimal_thresholds[method]
        
        # Filter objects based on the optimal confidence threshold
        filtered_objects = [obj for obj in inference_result[method]["objects"] if obj["confidence"] >= optimal_threshold]
        
        # Create a new inference result with the filtered objects
        optimized_result[method] = {
            "image_id": inference_result[method]["image_id"],
            "objects": filtered_objects,
            "inference_time": inference_result[method]["inference_time"],
            "upload_time": inference_result[method]["upload_time"],
            #"image_iou": inference_result[method]["image_iou"]
        }
    
    optimized_inference_results.append(optimized_result)

In [ ]:
# Calculates IoU for each image
# Iterate over each inference result
asd = 0
for inference_result in optimized_inference_results:
    asd +=1
    # Extract the image name from the inference result
    image_file = os.path.basename(inference_result["yolov5"]["image_id"])
    
    # Find the corresponding bbox data for the image
    bbox_data_item = next((data for data in original_bbox_data if data["image_id"] == image_file), None)
    
    # Calculate IoU for each inference method
    for method in inference_result:
        if bbox_data_item is not None:
            image = cv2.imread(inference_result[method]["image_id"])
            width, height = image.shape[:2]
            image_iou = calculate_iou(inference_result[method]["objects"], bbox_data_item["objects"],width, height)
            inference_result[method]["image_iou"] = image_iou
        else:
            print(f"No corresponding bbox data found for image: {image_file}")
    
    # Load and display the image with bounding boxes
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)
    
    for method in inference_result:
        print(method)
        print("Inference time:", inference_result[method]["inference_time"], "ms")
        print("IoU: ", inference_result[method]["image_iou"])
        draw_bounding_boxes(image_path, inference_result[method]["objects"])
    
    # Display the original image with the original bboxes
    print("Original Image:")
    draw_bounding_boxes(image_path, bbox_data_item["objects"])

In [ ]:
def output_results(content):
    print(content)
    text_file.write(content + "\n")

# Get hardware information
def get_cpu_info():
    cpu_info = cpuinfo.get_cpu_info()
    brand = cpu_info['brand_raw']
    return brand

# Get image resolution
image_path = optimized_inference_results[0]['yolov5']['image_id']  # all images have the same resolution
image = cv2.imread(image_path)
height, width, _ = image.shape
image_resolution = f"{width}x{height}"

# Get hardware information
cpu_info = get_cpu_info()
ram_info = psutil.virtual_memory()
try:
    gpu_info = gpustat.new_query()
except (ImportError, FileNotFoundError, Exception):
    gpu_info = False

# Calculate average inference time and image IoU for each method
method_metrics = {}
for inference_result in optimized_inference_results:
    for method in inference_result:
        if method not in method_metrics:
            method_metrics[method] = {'inference_time': [], 'upload_time': [], 'image_iou': []}
        method_metrics[method]['inference_time'].append(float(inference_result[method]['inference_time']))
        method_metrics[method]['upload_time'].append(float(inference_result[method]['upload_time']))
        method_metrics[method]['image_iou'].append(float(inference_result[method]['image_iou']))

for method in method_metrics:
    method_metrics[method]['avg_inference_time'] = np.mean(method_metrics[method]['inference_time'])
    method_metrics[method]['avg_upload_time'] = np.mean(method_metrics[method]['upload_time'])
    method_metrics[method]['avg_image_iou'] = np.mean(method_metrics[method]['image_iou'])

# Create a folder with the current datetime as its name
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"results_{current_datetime}"
folder_path = os.path.join(output_path, folder_name)
os.makedirs(folder_path, exist_ok=True)

# Convert numpy.float32 values to regular float objects
all_inference_results = json.loads(json.dumps(all_inference_results, default=lambda x: float(x) if isinstance(x, np.float32) else x))

# Save the original all_inference_results data as a JSON file
json_file_path = os.path.join(folder_path, "all_inference_results.json")
with open(json_file_path, "w") as json_file:
    json.dump(all_inference_results, json_file, indent=4)

# Save the printed information as a text file
text_file_path = os.path.join(folder_path, "results.txt")
with open(text_file_path, "w") as text_file:
    output_results("UPLOAD INFERENCE")
    output_results(f"Number of Images Tested: {len(optimized_inference_results)}")
    output_results(f"Image resolution: {image_resolution}")
    output_results(f"Dataset used: {dataset_name}")
    output_results(f"Dataset path: {image_folder}")
    output_results("SaaS region/server data:")
    output_results(f"AWS_REGION: {AWS_REGION}")
    output_results(f"GOOGLE_PROJECT_ID: {GOOGLE_PROJECT_ID}")
    output_results(f"MICROSOFT_ENDPOINT: {MICROSOFT_ENDPOINT}")
    output_results("Hardware Information:")
    output_results(f"CPU: {cpu_info}")
    output_results(f"RAM: {ram_info.total / (1024 * 1024 * 1024):.2f} GB")
    if (gpu_info):
        output_results(f"GPU: {gpu_info.name} - {gpu_info.memory_total / 1024:.2f} GB")
    else:
        output_results("GPU: N/A")
    output_results("Averaged Results:")
    for method, metrics in method_metrics.items():
        output_results(f"{method}:")
        output_results(f" Average Inference Time: {metrics['avg_inference_time']:.2f} ms")
        output_results(f" Average Upload Time: {metrics['avg_upload_time']:.2f} ms")
        output_results(f" Average Image IoU: {metrics['avg_image_iou']:.4f}")
        output_results(f" Optimal confidence threshold used: {optimal_thresholds[method]:.2f}")


In [ ]:
# Prepare data for plotting
methods = list(method_metrics.keys())
avg_inference_times = [method_metrics[method]['avg_inference_time'] for method in methods]
avg_upload_times = [method_metrics[method]['avg_upload_time'] for method in methods]
avg_image_ious = [method_metrics[method]['avg_image_iou'] for method in methods]

# Create and save the average inference time and upload time plot
plt.figure(figsize=(10, 6))
bar_width = 0.35
x = np.arange(len(methods))
plt.bar(x, avg_inference_times, bar_width, label='Inference Time')
plt.bar(x, avg_upload_times, bar_width, bottom=avg_inference_times, label='Upload Time')
plt.xlabel('Method')
plt.ylabel('Time (ms)')
plt.title('Average Inference Time and Upload Time by Method')
plt.xticks(x, methods, rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(folder_path, "avg_inference_upload_time.png"))
plt.show()
plt.close()

# Create and save the average image IoU plot
plt.figure(figsize=(10, 6))
plt.bar(methods, avg_image_ious)
plt.xlabel('Method')
plt.ylabel('Average Image IoU')
plt.title('Average Image IoU by Method')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(folder_path, "avg_image_iou.png"))
plt.show()
plt.close()

# Scatter charts showing how the average changes for both inference and accuracy over each inference call
inference_times = {method: [] for method in methods}
image_ious = {method: [] for method in methods}

for inference_result in optimized_inference_results:
    for method in methods:
        inference_times[method].append(float(inference_result[method]['inference_time']))
        image_ious[method].append(float(inference_result[method]['image_iou']))

# Combined line and scatter plots for inference time
plt.figure(figsize=(12, 6))
for method in methods:
    plt.plot(range(1, len(inference_times[method]) + 1), inference_times[method], label=method)
    plt.scatter(range(1, len(inference_times[method]) + 1), inference_times[method])
plt.xlabel('Inference Call')
plt.ylabel('Inference Time (ms)')
plt.title('Inference Time')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(folder_path, "inference_time.png"))
plt.show()
plt.close()

# Combined line and scatter plots for image IoU
plt.figure(figsize=(12, 6))
for method in methods:
    plt.plot(range(1, len(image_ious[method]) + 1), image_ious[method], label=method)
    plt.scatter(range(1, len(image_ious[method]) + 1), image_ious[method])
plt.xlabel('Inference Call')
plt.ylabel('Image IoU')
plt.title('Image IoU')
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(folder_path, "image_iou.png"))
plt.show()
plt.close()

# Histograms to visualize variance of confidence values, inference times, and IoU throughout each inference of image
confidence_values = {method: [] for method in methods}
inference_times_hist = {method: [] for method in methods}
image_ious_hist = {method: [] for method in methods}

for inference_result in optimized_inference_results:
    for method in methods:
        for obj in inference_result[method]['objects']:
            confidence_values[method].append(obj['confidence'])
        inference_times_hist[method].append(float(inference_result[method]['inference_time']))
        image_ious_hist[method].append(float(inference_result[method]['image_iou']))

# Confidence Value Distribution
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
axs = axs.flatten()
for i, method in enumerate(methods):
    axs[i].hist(confidence_values[method], bins=np.linspace(0, 1, 21), density=True)
    axs[i].set_xlabel('Confidence Value')
    axs[i].set_ylabel('Frequency')
    axs[i].set_title(f'Confidence Value Distribution - {method}')
plt.tight_layout()
plt.savefig(os.path.join(folder_path, "confidence_value_distribution.png"))
plt.show()
plt.close()

# Inference Time Distribution
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
axs = axs.flatten()
max_inference_time = max(max(times) for times in inference_times_hist.values())
bins = np.linspace(0, max_inference_time, 21)
for i, method in enumerate(methods):
    axs[i].hist(inference_times_hist[method], bins=bins, density=True)
    axs[i].set_xlabel('Inference Time (ms)')
    axs[i].set_ylabel('Frequency')
    axs[i].set_title(f'Inference Time Distribution - {method}')
plt.tight_layout()
plt.savefig(os.path.join(folder_path, "inference_time_distribution.png"))
plt.show()
plt.close()

# Image IoU Distribution
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
axs = axs.flatten()
for i, method in enumerate(methods):
    axs[i].hist(image_ious_hist[method], bins=np.linspace(0, 1, 21), density=True)
    axs[i].set_xlabel('Image IoU')
    axs[i].set_ylabel('Frequency')
    axs[i].set_title(f'Image IoU Distribution - {method}')
plt.tight_layout()
plt.savefig(os.path.join(folder_path, "image_iou_distribution.png"))
plt.show()
plt.close()

In [ ]:
single_test_path_image = "G:\\My Drive\\Thesis2024\\data\\train\\1478897977253835405_jpg.rf.k1YHtF480ieDxDfcboNr.jpg"
#test_aio = inferenceFromAllSources(single_test_path_image)

In [ ]:
for method in test_aio:
        print(method)
        print("Inference time:", test_aio[method]["inference_time"], "ms")
        #print("IoU: ", inference_result[method]["image_iou"])
        draw_bounding_boxes(single_test_path_image, test_aio[method]["objects"])

In [ ]:
# Load a saved file
# Specify the path to the JSON file containing the saved inference results
json_file_path = os.path.join(folder_path, "all_inference_results.json")

# Read the JSON file and load the data into the all_inference_results variable
with open(json_file_path, "r") as json_file:
    all_inference_results = json.load(json_file)

# Print the loaded data to verify it has been read correctly
print("Loaded data from JSON file:")
print(all_inference_results)